In [80]:
import pandas as pd
import numpy as np
import pickle

In [107]:
all_companies = pd.read_csv('AllCompanies_October_w_sector.csv')

In [66]:
company_sectors = pd.read_csv('company_sectors.csv')

In [67]:
all_companies = all_companies.merge(company_sectors[["symbol","sector"]], on="symbol")

In [9]:
with open('company_metrics_dict.pickle', 'rb') as handle:
    company_metrics = pickle.load(handle)

In [62]:
# prior year change in epd and revenue, past 5 years
percent_change_df = pd.DataFrame()
for key in company_metrics.keys():
    df = company_metrics[key]
    df = df.dropna(subset=["eps","revenue"]).sort_values(by='year')
    df['pct_change_eps'] = df['eps'].pct_change()
    df['pct_change_revenue'] = df['revenue'].pct_change()
    if len(df) > 5:
        percent_change_last_year_eps = df.loc[:, 'pct_change_eps'].iloc[-1]
        percent_change_last_year_rev = df.loc[:, 'pct_change_revenue'].iloc[-1]
        percent_change_last_5year_eps = df.loc[:, 'pct_change_eps'].iloc[-5:].mean()
        percent_change_last_5year_rev = df.loc[:, 'pct_change_revenue'].iloc[-5:].mean()
    elif len(df) > 2:
        percent_change_last_year_eps = df.loc[:, 'pct_change_eps'].iloc[-1]
        percent_change_last_year_rev = df.loc[:, 'pct_change_revenue'].iloc[-1]
        percent_change_last_5year_eps = df.loc[:, 'pct_change_eps'].iloc[-len(df):].mean()
        percent_change_last_5year_rev = df.loc[:, 'pct_change_revenue'].iloc[-len(df):].mean()
    else:
        continue
    percent_change_df = percent_change_df.append(pd.Series({"symbol": key, 
                                                            "percent_change_last_year_eps": percent_change_last_year_eps,
                                                           "percent_change_last_year_rev": percent_change_last_year_rev,
                                                           "percent_change_last_5year_eps": percent_change_last_5year_eps,
                                                           "percent_change_last_5year_rev": percent_change_last_5year_rev}), ignore_index=True)
    

In [68]:
all_companies = all_companies.merge(percent_change_df, on="symbol")

In [72]:
all_companies.columns

Index(['symbol', 'name', 'price', 'income_statement_date', 'ROC',
       'EarningsYield', 'irr', 'npv_mean', 'npv_regression', 'regression_type',
       'eps_roc', 'eps_roc_regression', 'ROE', 'eps_diluted', 'MOP', 'QA',
       'MCap', 'eps_roc_flag', 'eps_base', 'eps_list', 'error_message',
       'industry', 'dividend_ratio', 'dividend_trend',
       'commonstock_repurchased_trend', 'commonstock_repurchased', 'PE',
       'sector', 'percent_change_last_year_eps',
       'percent_change_last_year_rev', 'percent_change_last_5year_eps',
       'percent_change_last_5year_rev'],
      dtype='object')

In [76]:
# Only include companies > 1bn MCAP
sector_analysis_companies = all_companies[all_companies.MCap > 10**9]

In [81]:
sector_analysis_companies = sector_analysis_companies.replace(np.inf, 0)

In [82]:
sector_analysis_companies = sector_analysis_companies.dropna(subset=["sector","percent_change_last_year_eps",
                                                                     "percent_change_last_year_rev",
                                                                     "percent_change_last_5year_eps",
                                                                     "percent_change_last_5year_rev",
                                                                    "ROC", "EarningsYield","dividend_ratio"])

In [102]:
from portfolio_analysis.data import get_all_company_tickers
from portfolio_analysis.api import apikey

In [112]:
# company_names = get_all_company_tickers(apikey)
# sector_analysis_companies = sector_analysis_companies.merge(company_names[["symbol", "exchange"]], on="symbol")
all_companies = all_companies.merge(company_names[["symbol", "exchange"]], on="symbol")

In [105]:
sector_analysis = sector_analysis_companies[["sector","exchange","percent_change_last_year_eps",
                                                                     "percent_change_last_year_rev",
                                                                     "percent_change_last_5year_eps",
                                                                     "percent_change_last_5year_rev",
                                                                    "ROC", "EarningsYield","dividend_ratio"]].groupby(["sector","exchange"]).agg("mean").reset_index()

In [106]:
sector_analysis.to_csv('sector_analysis.csv')

In [110]:
all_companies = all_companies.drop(columns=['percent_change_last_year_eps', 'percent_change_last_year_rev',
       'percent_change_last_5year_eps', 'percent_change_last_5year_rev',
       'percent_change_last_year_eps_sector',
       'percent_change_last_year_rev_sector',
       'percent_change_last_5year_eps_sector',
       'percent_change_last_5year_rev_sector', 'ROC_sector',
       'EarningsYield_sector', 'dividend_ratio_sector',])

In [113]:
acws = all_companies.merge(sector_analysis, on=["sector", "exchange"], suffixes=("","_sector"))

In [114]:
acws.to_csv("AllCompanies_October_w_sector.csv")

In [115]:
mf_companies = pd.read_csv('mf_company_analysis.csv')

In [116]:
mf_companies = mf_companies.merge(company_names[["symbol", "exchange"]], on="symbol")

In [117]:
mf_companies = mf_companies.merge(sector_analysis, on=["sector", "exchange"], suffixes=("","_sector"))

In [118]:
mf_companies.to_csv("mf_company_analysis.csv")

In [120]:
sector_analysis.to_csv('sector_analysis.csv')